In [1]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 26.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 72.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2
  Attempting uninstall: python-multipart
    Found existing installation: python-multipart 0.0.9
    Uninstalling python-multipart-0.0.9:
      Successfully uninstalled python-multipart-0.0.9
  Attempting uninstall: starlette
    Found existing installation: starlette 0.37.2
    Uninstalling starlette-0.37.2:
      Successfully uninstalled starlette-0.37.2
  Attempting uninstall: fastapi
    Found existing installation: f

## Loading all the models

### VGG16 Model

In [2]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

import torch
import torch.nn as nn
import torchvision.models as models

model_name = 'vgg16'
#Customvgg16 class inherit from nn.Module
class CustomVGG16(nn.Module):
    def __init__(self, num_classes=2):
        super(CustomVGG16, self).__init__()

        # Loading pre-trained VGG16 model from tourchvision.models
        vgg16 = models.vgg16(pretrained=True)

        # Extracting the features and avgpool layers frm pretrainedmodel
        self.features = vgg16.features
        self.avgpool = vgg16.avgpool

        # Define a new classifier nm.Sequential
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096), #input and output
            nn.ReLU(inplace=True), #for activation
            nn.Dropout(), #for regularization
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):

        x = self.features(x) # Passing the input through the features layer
        x = self.avgpool(x) # Using the avgpool layer
        x = torch.flatten(x, 1) # Reshaping the output to a 2D tensor
        x = self.classifier(x) # Passing the reshaped output to the custom classifier
        return x


### Resnet18 Model

In [3]:
class Resnet18(nn.Module):

    def __init__(self, num_classes=2):
        super(Resnet18, self).__init__()

        model_resnet18 = models.resnet18(pretrained=True)

        self.conv1 = model_resnet18.conv1  # convolutional function
        self.bn1 = model_resnet18.bn1  # batch normalization
        self.relu = model_resnet18.relu  # relu is your activation function.
        self.maxpool = model_resnet18.maxpool  # maxpool is basically taking the biggest value per sub-matrix

        self.layer1 = model_resnet18.layer1
        self.layer2 = model_resnet18.layer2
        self.layer3 = model_resnet18.layer3
        self.layer4 = model_resnet18.layer4  # these layers are use for deepening the layers in the architecture which will increase

        self.avgpool = model_resnet18.avgpool
        self._features = model_resnet18.fc.in_features

        self.fc = nn.Linear(self._features, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

### restnet50 Model

In [4]:
class Resnet50(nn.Module):
    def __init__(self, num_classes=2):
        super(Resnet50, self).__init__()

        model_resnet50 = models.resnet50(pretrained=True)

        self.conv1 = model_resnet50.conv1
        self.bn1 = model_resnet50.bn1
        self.relu = model_resnet50.relu
        self.maxpool = model_resnet50.maxpool

        self.layer1 = model_resnet50.layer1
        self.layer2 = model_resnet50.layer2
        self.layer3 = model_resnet50.layer3
        self.layer4 = model_resnet50.layer4

        self.avgpool = model_resnet50.avgpool
        self._features = model_resnet50.fc.in_features

        self.fc = nn.Linear(self._features, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)

        x = self.fc(x)

        return x

### Loading all the models 

In [5]:
#The pre-trained weights from vgg16.pth, restnet18.pth, resnet50 are loaded into the model.

vgg16_path = '/kaggle/input/tumor-trace-models/kaggle/VGG16_Model/vgg16_best.pth'
resnet18_path = '/kaggle/input/tumor-trace-models/kaggle/Resnet18_Model/resnet18_best.pth'
resnet50_path = '/kaggle/input/tumor-trace-models/kaggle/Resnet50_Model/resnet50_best.pth'

vgg16_model = CustomVGG16(num_classes=2)
vgg16_model.load_state_dict(torch.load(vgg16_path, map_location = torch.device('cpu')))
vgg16_model.eval() #model is set to evaluation


resnet18_model = Resnet18(num_classes=2)
resnet18_model.load_state_dict(torch.load(resnet18_path, map_location=torch.device('cpu')))
resnet18_model.eval()

resnet50_model = Resnet50(num_classes=2)
resnet50_model.load_state_dict(torch.load(resnet50_path, map_location=torch.device('cpu')))
resnet50_model.eval()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 163MB/s] 
/tmp/ipykernel_30/3656547092.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to constr

Resnet50(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1

### Defining the Preprocessing

In [6]:
# Preprocessing the input image to match your modelss' requirements.

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

### Prediction Function

In [7]:
def predict(model_name, image):
    #preprocessing the image
    # adds a new dimension at the 0th position (i.e., the first position).
    img = preprocess(image).unsqueeze(0) # This sets the batch size to 1-> [1, channels, height, width]

    # Based on the user's selection (model_name), the corresponding model is chosen.
    if model_name == "VGG16":
        model = vgg16_model
    elif model_name == "ResNet18":
        model = resnet18_model
    elif model_name == "ResNet50":
        model = resnet50_model
    else:
        return "Invalid model name"

    
    # Get the model's prediction
    with torch.no_grad(): #disable gradient calculation 
        outputs = model(img)
        _, predicted = torch.max(outputs, 1)  #selects the class with the highest probability.
    
    # Define class labels
    classes = ["benign", "malignant"]
    return classes[predicted.item()]  # Return the predicted label




### Creating Interface

In [8]:
import gradio as gr

# Custom HTML for title and description
with gr.Blocks() as block:
    with gr.Row():
        gr.HTML("""
            <div style="text-align: center;">
                <h1 style="color: #ffffff;">TumorTrace: MRI-Based AI for Breast Cancer Detection</h1>
                <p style="color: #c4c4c4;">Select a model and upload an MRI image to classify it as benign or malignant.</p>
            </div>
        """)

# Custom CSS for background and button styling
css = """
<style>
body {
    background-image: url('/kaggle/input/background-images/preview.jpg');
    background-size: cover;
    background-repeat: no-repeat;
    background-attachment: fixed;
}

.gr-button {
    background-color: #f76c6c;
    color: #ffffff;
    border: none;
}

.gr-button:hover {
    background-color: #ff4c4c;
}
</style>
"""

# Interface
interface = gr.Interface(
    fn=predict,  # predict function to be called
    inputs=[
        gr.Dropdown(choices=["VGG16", "ResNet18", "ResNet50"], label="Select Model"),
        gr.Image(type="pil")
    ],
    outputs="label",
    title="Tumor Classification",
    description="Select a model and upload an MRI image to classify it as benign or malignant.",
    css=css  # Apply CSS to the interface
)

# Launch the app
interface.launch()

* Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://6af675860410b85f4a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
